In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../src')

from plgUtil import *
from genAlgo import *
from faustGen import *

from pedalboard import Pedalboard, Plugin, Chain, Mix, Gain, Chorus, Reverb
from pedalboard.io import AudioFile

In [5]:
# Read in a whole audio file:
with AudioFile('guitar.wav', 'r') as f:
    audio = f.read(f.frames)
    samplerate = f.samplerate

# Make a Pedalboard object, containing multiple plugins:
board = Pedalboard([Reverb(), Mix([Chorus(), Reverb()]), Gain(gain_db=0.9)])

# Run the audio through this pedalboard!
effected = board(audio, samplerate)

# Write the audio back as a wav file:
with AudioFile('./test4_revmix/guitar_rev_chorrev_gain.wav', 'w', samplerate, effected.shape[0]) as f:
    f.write(effected)

In [2]:
from IPython.display import Audio
# from glob import glob
# audio = glob('./*')

Audio('./guitar.wav')

In [4]:
Audio('./test2_reverb/guitar_rev.wav')

In [2]:
with AudioFile('guitar.wav', 'r') as f:
    dry = f.read(f.frames)
    sr = f.samplerate
with AudioFile('./test4_revmix/guitar_rev_chorrev_gain.wav', 'r') as f:
    wet = f.read(f.frames)

In [3]:
evolve(dry, wet, sr)

/home/sam/anaconda3/lib/python3.9/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 2048 is greater than input length  = 2, using nperseg = 2
  warnings.warn('nperseg = {0:d} is greater than input length '


BEST MODEL THUS FAR (0)
 ERROR: 0.0003622657968662679
 MODEL: <Pedalboard with 1 plugin: [<pedalboard.Chain with 1 plugin: [<pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x55e410af3060>] at 0x55e410992010>]>
BEST MODEL THUS FAR (1)
 ERROR: 0.0003622657968662679
 MODEL: <Pedalboard with 1 plugin: [<pedalboard.Chain with 1 plugin: [<pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x55e410af3060>] at 0x55e410992010>]>
BEST MODEL THUS FAR (2)
 ERROR: 0.0003622657968662679
 MODEL: <Pedalboard with 1 plugin: [<pedalboard.Chain with 1 plugin: [<pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x55e410af3060>] at 0x55e410992010>]>
BEST MODEL THUS FAR (3)
 ERROR: 0.0003622657968662679
 MODEL: <Pedalboard with 1 plugin: [<pedalboard.Chain with 1 plugin: [<pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x55e410af3060>] at 0x55e410992010>]>
BEST MODEL THUS FAR (4)
 ERROR: 


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         2 variables are exactly at the bounds

At iterate    0    f=  7.10993D-04    |proj g|=  2.44472D-03

At iterate    1    f=  7.03053D-04    |proj g|=  2.09548D-03

At iterate    2    f=  7.00878D-04    |proj g|=  9.37143D-03
  ys=-3.726E-07  -gs= 3.821E-06 BFGS update SKIPPED

At iterate    3    f=  7.00873D-04    |proj g|=  2.56114D-03

At iterate    4    f=  6.99748D-04    |proj g|=  1.62981D-03

At iterate    5    f=  6.87481D-04    |proj g|=  3.56813D-02
  ys=-7.601E-06  -gs= 1.388E-05 BFGS update SKIPPED

At iterate    6    f=  6.85281D-04    |proj g|=  3.20142D-03

At iterate    7    f=  6.85125D-04    |proj g|=  1.68802D-03

At iterate    8    f=  6.43266D-04    |proj g|=  1.00000D+00
  ys=-4.103E-03  -gs= 1.177E-03 BFGS update SKIPPED

At iterate    9    f=  6.41991D-04    |proj g|=  4.59841D-03

At iterate   10    f=  6.02216D-04    |proj g|=  1.19500

<Pedalboard with 5 plugins: [<pedalboard.Chain with 1 plugin: [<pedalboard.Reverb room_size=0.560364 damping=0.633043 wet_level=0.228599 dry_level=0.180937 width=0.993301 freeze_mode=0.136155 at 0x55e411540970>] at 0x55e4115a9070>, <pedalboard.Chain with 1 plugin: [<pedalboard.Gain gain_db=0 at 0x55e410848b20>] at 0x55e410848ba0>, <pedalboard.Chain with 1 plugin: [<pedalboard.Gain gain_db=0 at 0x55e4106c0150>] at 0x55e411442d90>, <pedalboard.Chain with 1 plugin: [<pedalboard.Chorus rate_hz=1 depth=0.251439 centre_delay_ms=7.00268 feedback=3.63741e-05 mix=0.499538 at 0x55e4116a61a0>] at 0x55e41145b820>, <pedalboard.Chain with 1 plugin: [<pedalboard.Reverb room_size=0.560364 damping=0.633043 wet_level=0.228599 dry_level=0.180937 width=0.993301 freeze_mode=0.136155 at 0x55e410cb6d90>] at 0x55e410984670>]>

In [6]:
# initialize models
models = [Pedalboard([Chain([newPlg(plg)])]) for plg in PLUGINS]

# do I want this here?
models = generation(models, N_pop, N_survive)

gen_num = 0
converge = False
elapsed = ti.time()


while gen_num < N_gens :

    errors = [calc_error(model(dry, sr), wet) for model in models]
    aridxs = np.argsort(errors)[:N_pop]
    errors = [errors[i] for i in aridxs]
    models = [models[i] for i in aridxs]

    if errors[0] <= tol :
        converge = True
        break

    if N_pop < 48:
        N_pop += 4

    models = generation(models, N_pop, N_survive)

conv = "converged" if converge else "did not converge"
print(f"The model {conv}!")
print(f"Best Model: {models[0]} \nError: {models[0]}")




PARENT IN MODEL: False
MODELS (type=<class 'list'>): [<Pedalboard with 1 plugin: [<pedalboard.Chain with 1 plugin: [<pedalboard.Reverb room_size=0.5 damping=0.5 wet_level=0.33 dry_level=0.4 width=1 freeze_mode=0 at 0x55601d7e0290>] at 0x55601d8315d0>]>, <Pedalboard with 1 plugin: [<pedalboard.Chain with 1 plugin: [<pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x55601d865f10>] at 0x55601d81c930>]>, <Pedalboard with 1 plugin: [<pedalboard.Chain with 1 plugin: [<pedalboard.Gain gain_db=1 at 0x55601d7b9730>] at 0x55601d81a460>]>]

    PARENTS: [<Pedalboard with 1 plugin: [<pedalboard.Chain with 1 plugin: [<pedalboard.Reverb room_size=0.5 damping=0.5 wet_level=0.33 dry_level=0.4 width=1 freeze_mode=0 at 0x55601d6eac10>] at 0x55601c352fb0>]>, <Pedalboard with 1 plugin: [<pedalboard.Chain with 1 plugin: [<pedalboard.Reverb room_size=0.5 damping=0.5 wet_level=0.33 dry_level=0.4 width=1 freeze_mode=0 at 0x55601d7766c0>] at 0x55601d1b9290>]>]

CONCAT PARENTS in 

/home/sam/anaconda3/lib/python3.9/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 2048 is greater than input length  = 2, using nperseg = 2
  warnings.warn('nperseg = {0:d} is greater than input length '


The model converged!
Best Model: <Pedalboard with 1 plugin: [<pedalboard.Chain with 1 plugin: [<pedalboard.Reverb room_size=0.5 damping=0.5 wet_level=0.33 dry_level=0.4 width=1 freeze_mode=0 at 0x55601d7e0290>] at 0x55601d8315d0>]> 
Error: <Pedalboard with 1 plugin: [<pedalboard.Chain with 1 plugin: [<pedalboard.Reverb room_size=0.5 damping=0.5 wet_level=0.33 dry_level=0.4 width=1 freeze_mode=0 at 0x55601d7e0290>] at 0x55601d8315d0>]>


In [9]:
models

[<Pedalboard with 1 plugin: [<pedalboard.Chain with 1 plugin: [<pedalboard.Reverb room_size=0.5 damping=0.5 wet_level=0.33 dry_level=0.4 width=1 freeze_mode=0 at 0x558b78ca6e50>] at 0x558b7880e120>]>,
 <Pedalboard with 1 plugin: [<pedalboard.Chain with 1 plugin: [<pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x558b78cc0e20>] at 0x558b78743c10>]>,
 <Pedalboard with 1 plugin: [<pedalboard.Chain with 1 plugin: [<pedalboard.Gain gain_db=1 at 0x558b7882a000>] at 0x558b7875cdb0>]>,
 <Pedalboard with 3 plugins: [<pedalboard.Chain with 1 plugin: [<pedalboard.Reverb room_size=0.5 damping=0.5 wet_level=0.33 dry_level=0.4 width=1 freeze_mode=0 at 0x558b78cda680>] at 0x558b786e1230>, <pedalboard.Chain with 1 plugin: [<pedalboard.Reverb room_size=0.5 damping=0.5 wet_level=0.33 dry_level=0.4 width=1 freeze_mode=0 at 0x558b78cc17f0>] at 0x558b786f3060>, <pedalboard.Chain with 1 plugin: [<pedalboard.Gain gain_db=1 at 0x558b77e13970>] at 0x558b7880e0c0>]>,
 <Pedalboard

In [12]:
for i in range(len(models)) :
    if not (isinstance(models[i], Chain) or isinstance(models[i], Mix)) :
        print(f"type: {type(Chain([models[i]]))}")
        models[i] = Chain([models[i]])
        print(f"THIS FUCKING MODEL: {models[i]}")
        models[i] = Pedalboard([Chain([models[i]])])

In [14]:
for _ in range(20) :
    if None in [newBoard(rd.choice(models[:N_survive])) for _ in range(2)] :
        print("AHHHHHHHHHHHHHHHHHH", models)

TypeError: 'NoneType' object is not iterable

In [12]:
a = 1
b = [2, a, 3]
b.remove(a)
b

[2, 3]